# WineMood – Classificador de Sentimentos para Avaliações de Vinhos

Este notebook implementa um sistema de classificação de sentimentos para avaliações de vinhos usando técnicas de Processamento de Linguagem Natural (NLP) e Deep Learning.

## Objetivos
- Analisar avaliações textuais de vinhos
- Classificar o sentimento como positivo, neutro ou negativo
- Criar um modelo de Deep Learning para entender o contexto das avaliações
- Desenvolver uma interface web para teste em tempo real

## 1. Configuração do Ambiente

Primeiro, vamos instalar e importar as bibliotecas necessárias.

In [ ]:
# Instalar bibliotecas necessárias
!pip install kaggle nltk tensorflow pandas numpy matplotlib streamlit

In [ ]:
# Importar bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import pickle

# Bibliotecas para NLP
import nltk
from nltk.corpus import stopwords

# Bibliotecas para Deep Learning
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Configurar visualização
plt.style.use('ggplot')
sns.set(style='whitegrid')

# Verificar se GPU está disponível
print("GPU disponível:", tf.config.list_physical_devices('GPU'))

## 2. Download e Carregamento do Dataset

Vamos baixar o dataset de avaliações de vinhos do Kaggle.

In [ ]:
# Configurar a API do Kaggle
# Nota: Você precisa fazer upload do arquivo kaggle.json para o Colab
from google.colab import files
uploaded = files.upload()

# Criar diretório para as credenciais do Kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Baixar o dataset
!kaggle datasets download -d zynicide/wine-reviews
!unzip wine-reviews.zip

In [ ]:
# Carregar o dataset
df = pd.read_csv('winemag-data-130k-v2.csv')

# Exibir as primeiras linhas
df.head()

## 3. Exploração e Pré-processamento de Dados

Vamos explorar o dataset e preparar os dados para o treinamento.

In [ ]:
# Verificar informações do dataset
print("Formato do dataset:", df.shape)
print("\nInformações do dataset:")
df.info()

# Verificar valores nulos
print("\nValores nulos por coluna:")
print(df.isnull().sum())

In [ ]:
# Verificar estatísticas descritivas
print("Estatísticas descritivas da pontuação:")
df['points'].describe()

In [ ]:
# Visualizar a distribuição de pontuações
plt.figure(figsize=(12, 6))
sns.histplot(df['points'], bins=20, kde=True)
plt.title('Distribuição de Pontuações de Vinhos')
plt.xlabel('Pontuação')
plt.ylabel('Frequência')
plt.grid(True)
plt.show()

In [ ]:
# Filtrar apenas as colunas necessárias e remover valores nulos
df_filtered = df[['description', 'points']].dropna()
print(f"Formato após filtragem: {df_filtered.shape}")

In [ ]:
# Criar rótulos de sentimento baseados na pontuação
def create_sentiment_label(points):
    if points >= 90:
        return 2  # Positivo
    elif points >= 80:
        return 1  # Neutro
    else:
        return 0  # Negativo

df_filtered['sentiment'] = df_filtered['points'].apply(create_sentiment_label)

# Verificar a distribuição de sentimentos
sentiment_counts = df_filtered['sentiment'].value_counts()
print("Distribuição de sentimentos:")
print(sentiment_counts)

# Visualizar a distribuição
plt.figure(figsize=(10, 6))
sns.countplot(x='sentiment', data=df_filtered, palette=['red', 'gray', 'green'])
plt.title('Distribuição de Sentimentos')
plt.xlabel('Sentimento (0: Negativo, 1: Neutro, 2: Positivo)')
plt.ylabel('Contagem')
plt.grid(True)
plt.show()

### Pré-processamento de Texto

Vamos limpar e preparar o texto para o modelo.

In [ ]:
# Baixar stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [ ]:
# Função para pré-processar o texto
def preprocess_text(text):
    # Converter para minúsculas
    text = text.lower()
    
    # Remover pontuação e caracteres especiais
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remover stopwords
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    
    return ' '.join(filtered_words)

# Aplicar pré-processamento
df_filtered['processed_text'] = df_filtered['description'].apply(preprocess_text)

# Exibir exemplos de texto original e processado
examples = df_filtered[['description', 'processed_text', 'sentiment']].sample(5)
examples

## 4. Preparação dos Dados para o Modelo

Vamos tokenizar e padronizar os textos para alimentar o modelo de Deep Learning.

In [ ]:
# Definir parâmetros
MAX_NUM_WORDS = 10000  # Tamanho do vocabulário
MAX_SEQUENCE_LENGTH = 100  # Comprimento máximo da sequência
EMBEDDING_DIM = 100  # Dimensão do embedding

# Criar tokenizer
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(df_filtered['processed_text'])

# Converter textos para sequências
sequences = tokenizer.texts_to_sequences(df_filtered['processed_text'])

# Padronizar sequências
padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

# Verificar formato dos dados
print(f"Formato das sequências padronizadas: {padded_sequences.shape}")

In [ ]:
# Preparar rótulos para classificação multiclasse
labels = tf.keras.utils.to_categorical(df_filtered['sentiment'], num_classes=3)
print(f"Formato dos rótulos: {labels.shape}")

In [ ]:
# Dividir em conjuntos de treino e teste (80/20)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, labels, test_size=0.2, random_state=42, stratify=labels
)

print(f"Conjunto de treino: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de teste: {X_test.shape}, {y_test.shape}")

## 5. Construção e Treinamento do Modelo

Vamos criar um modelo de Deep Learning com camadas de Embedding e LSTM.

In [ ]:
# Definir o modelo
model = Sequential()
model.add(Embedding(input_dim=MAX_NUM_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))  # 3 classes: negativo, neutro, positivo

# Compilar o modelo
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Resumo do modelo
model.summary()

In [ ]:
# Definir callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    'best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

In [ ]:
# Treinar o modelo
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=64,
    validation_split=0.1,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

## 6. Avaliação do Modelo

Vamos avaliar o desempenho do modelo no conjunto de teste.

In [ ]:
# Carregar o melhor modelo
best_model = tf.keras.models.load_model('best_model.h5')

# Avaliar no conjunto de teste
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=1)
print(f"Acurácia no conjunto de teste: {test_accuracy:.4f}")

In [ ]:
# Visualizar o histórico de treinamento
plt.figure(figsize=(12, 5))

# Gráfico de acurácia
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title('Acurácia do Modelo')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()
plt.grid(True)

# Gráfico de perda
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Perda do Modelo')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Fazer predições no conjunto de teste
y_pred_proba = best_model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis=1)
y_true = np.argmax(y_test, axis=1)

# Matriz de confusão
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Negativo', 'Neutro', 'Positivo'],
            yticklabels=['Negativo', 'Neutro', 'Positivo'])
plt.title('Matriz de Confusão')
plt.xlabel('Predito')
plt.ylabel('Real')
plt.show()

# Relatório de classificação
print("Relatório de Classificação:")
print(classification_report(y_true, y_pred, target_names=['Negativo', 'Neutro', 'Positivo']))

## 7. Análise de Exemplos Incorretos

Vamos analisar alguns exemplos que o modelo classificou incorretamente.

In [ ]:
# Identificar exemplos classificados incorretamente
incorrect_indices = np.where(y_pred != y_true)[0]
print(f"Número de exemplos classificados incorretamente: {len(incorrect_indices)}")

# Selecionar alguns exemplos incorretos aleatoriamente
if len(incorrect_indices) > 0:
    sample_size = min(5, len(incorrect_indices))
    sample_indices = np.random.choice(incorrect_indices, size=sample_size, replace=False)
    
    # Recuperar os textos originais
    original_texts = df_filtered['description'].values[np.where(df_filtered.index.isin(X_test[sample_indices]))[0]]
    
    # Exibir exemplos incorretos
    for i, idx in enumerate(sample_indices):
        true_label = ['Negativo', 'Neutro', 'Positivo'][y_true[idx]]
        pred_label = ['Negativo', 'Neutro', 'Positivo'][y_pred[idx]]
        print(f"Exemplo {i+1}:")
        print(f"Texto: {original_texts[i]}")
        print(f"Rótulo real: {true_label}")
        print(f"Rótulo predito: {pred_label}")
        print("---")

## 8. Salvar o Modelo e o Tokenizer

Vamos salvar o modelo treinado e o tokenizer para uso na aplicação web.

In [ ]:
# Salvar o modelo final
best_model.save('model.h5')
print("Modelo salvo como 'model.h5'")

# Salvar o tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("Tokenizer salvo como 'tokenizer.pickle'")

## 9. Teste do Modelo com Exemplos

Vamos testar o modelo com alguns exemplos de avaliações de vinhos.

In [ ]:
# Função para classificar o sentimento de um texto
def classify_sentiment(text, model, tokenizer):
    # Pré-processar o texto
    processed_text = preprocess_text(text)
    
    # Tokenizar e padronizar
    sequence = tokenizer.texts_to_sequences([processed_text])
    padded_sequence = pad_sequences(sequence, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
    
    # Fazer a predição
    prediction = model.predict(padded_sequence)
    sentiment_class = np.argmax(prediction, axis=1)[0]
    
    # Mapear para rótulos de sentimento
    sentiment_map = {0: "Negativo", 1: "Neutro", 2: "Positivo"}
    sentiment = sentiment_map[sentiment_class]
    
    # Obter as probabilidades
    probabilities = prediction[0]
    
    return sentiment, probabilities

In [ ]:
# Exemplos de avaliações de vinhos
examples = [
    "Great complexity with floral aromas and a long finish.",
    "This wine is average at best, with a short finish and little character.",
    "Terrible wine, tastes like vinegar and has no depth.",
    "Balanced acidity with notes of apple and pear, decent for the price.",
    "Exceptional depth and complexity, with layers of dark fruit and spice."
]

# Testar o modelo com os exemplos
for example in examples:
    sentiment, probabilities = classify_sentiment(example, best_model, tokenizer)
    print(f"Texto: {example}")
    print(f"Sentimento: {sentiment}")
    print(f"Probabilidades: Negativo: {probabilities[0]:.4f}, Neutro: {probabilities[1]:.4f}, Positivo: {probabilities[2]:.4f}")
    print("---")

## 10. Preparação para a Interface Web

Vamos preparar os arquivos necessários para a aplicação Streamlit.

In [ ]:
# Criar arquivo requirements.txt
requirements = """
streamlit==1.22.0
tensorflow==2.12.0
nltk==3.8.1
numpy==1.23.5
pandas==1.5.3
matplotlib==3.7.1
"""

with open('requirements.txt', 'w') as f:
    f.write(requirements.strip())
    
print("Arquivo requirements.txt criado")

In [ ]:
# Baixar arquivos para uso local
from google.colab import files

files.download('model.h5')
files.download('tokenizer.pickle')
files.download('requirements.txt')

## 11. Código da Aplicação Streamlit

Abaixo está o código para a aplicação Streamlit que pode ser salvo como `app.py`.

In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import nltk
from nltk.corpus import stopwords
import pickle

# Configuração da página
st.set_page_config(
    page_title="WineMood - Sentiment Analyzer",
    page_icon="🍷",
    layout="wide",
    initial_sidebar_state="expanded",
    menu_items={
        'About': "# WineMood - Analisador de Sentimentos para Avaliações de Vinhos"
    }
)

# Aplicar tema escuro personalizado
st.markdown("""
<style>
    .main {
        background-color: #1E1E1E;
        color: #FFFFFF;
    }
    .stTextInput > div > div > input {
        background-color: #2D2D2D;
        color: #FFFFFF;
    }
    .stButton>button {
        background-color: #8A2BE2;
        color: white;
        font-weight: bold;
        border-radius: 5px;
        padding: 0.5rem 1rem;
        border: none;
    }
    .stButton>button:hover {
        background-color: #9370DB;
    }
    .positive {
        color: #4CAF50;
        font-weight: bold;
        font-size: 24px;
    }
    .neutral {
        color: #9E9E9E;
        font-weight: bold;
        font-size: 24px;
    }
    .negative {
        color: #F44336;
        font-weight: bold;
        font-size: 24px;
    }
    .title {
        font-size: 42px;
        font-weight: bold;
        text-align: center;
        margin-bottom: 30px;
        color: #FFFFFF;
    }
    .subtitle {
        font-size: 24px;
        font-weight: bold;
        margin-bottom: 20px;
        color: #FFFFFF;
    }
</style>
""", unsafe_allow_html=True)

# Função para carregar o modelo e o tokenizer
@st.cache_resource
def load_model_and_tokenizer():
    try:
        # Carregar o modelo
        model = tf.keras.models.load_model('model.h5')
        
        # Carregar o tokenizer
        with open('tokenizer.pickle', 'rb') as handle:
            tokenizer = pickle.load(handle)
            
        return model, tokenizer
    except Exception as e:
        st.error(f"Erro ao carregar o modelo: {e}")
        return None, None

# Função para pré-processar o texto
def preprocess_text(text, tokenizer, max_length=100):
    # Converter para minúsculas
    text = text.lower()
    
    # Remover pontuação e caracteres especiais
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remover stopwords
    try:
        nltk.data.find('corpora/stopwords')
    except LookupError:
        nltk.download('stopwords', quiet=True)
    
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    # Tokenizar e padronizar
    sequences = tokenizer.texts_to_sequences([text])
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')
    
    return padded_sequences

# Função para classificar o sentimento
def classify_sentiment(text, model, tokenizer):
    # Pré-processar o texto
    processed_text = preprocess_text(text, tokenizer)
    
    # Fazer a predição
    prediction = model.predict(processed_text)
    
    # Obter a classe com maior probabilidade
    sentiment_class = np.argmax(prediction, axis=1)[0]
    
    # Mapear para rótulos de sentimento
    sentiment_map = {0: "Negativo", 1: "Neutro", 2: "Positivo"}
    sentiment = sentiment_map[sentiment_class]
    
    # Obter as probabilidades
    probabilities = prediction[0]
    
    return sentiment, probabilities

# Função para exibir o resultado com a cor apropriada
def display_sentiment(sentiment):
    if sentiment == "Positivo":
        return st.markdown(f'<p class="positive">Sentimento: {sentiment}</p>', unsafe_allow_html=True)
    elif sentiment == "Neutro":
        return st.markdown(f'<p class="neutral">Sentimento: {sentiment}</p>', unsafe_allow_html=True)
    else:
        return st.markdown(f'<p class="negative">Sentimento: {sentiment}</p>', unsafe_allow_html=True)

# Função para plotar o gráfico de barras
def plot_sentiment_probabilities(probabilities):
    labels = ['Negativo', 'Neutro', 'Positivo']
    
    fig, ax = plt.subplots(figsize=(10, 5))
    fig.patch.set_facecolor('#1E1E1E')
    ax.set_facecolor('#1E1E1E')
    
    bars = ax.bar(labels, probabilities * 100, color=['#F44336', '#9E9E9E', '#4CAF50'])
    
    # Adicionar rótulos e título
    ax.set_ylabel('Probabilidade (%)', color='white')
    ax.set_title('Distribuição de Probabilidades de Sentimento', color='white')
    
    # Personalizar eixos
    ax.spines['bottom'].set_color('white')
    ax.spines['top'].set_color('white')
    ax.spines['right'].set_color('white')
    ax.spines['left'].set_color('white')
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')
    
    # Adicionar valores nas barras
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.1f}%',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom',
                    color='white')
    
    return fig

# Interface principal
def main():
    # Carregar o modelo e o tokenizer
    model, tokenizer = load_model_and_tokenizer()
    
    # Título
    st.markdown('<p class="title">WineMood – Sentiment Analyzer</p>', unsafe_allow_html=True)
    
    # Criar abas
    tab1, tab2 = st.tabs(["Analisador de Sentimentos", "Sobre o Projeto"])
    
    with tab1:
        st.markdown('<p class="subtitle">Análise de Sentimentos em Avaliações de Vinhos</p>', unsafe_allow_html=True)
        
        # Área de texto para entrada do usuário
        user_input = st.text_area("Digite a descrição do vinho (ou várias descrições, uma por linha):", 
                                height=150,
                                placeholder="Ex: Great complexity with floral aromas and a long finish.")
        
        # Botão para classificar
        if st.button("Classify"):
            if user_input:
                # Dividir o texto em linhas para processar múltiplas avaliações
                reviews = user_input.strip().split('\n')
                
                # Processar cada avaliação
                results = []
                all_probabilities = []
                
                for review in reviews:
                    if review.strip():  # Verificar se a linha não está vazia
                        sentiment, probabilities = classify_sentiment(review, model, tokenizer)
                        results.append((review, sentiment))
                        all_probabilities.append(probabilities)
                
                # Exibir resultados
                if len(results) == 1:
                    # Caso de uma única avaliação
                    display_sentiment(results[0][1])
                    
                    # Plotar gráfico de probabilidades
                    st.pyplot(plot_sentiment_probabilities(all_probabilities[0]))
                else:
                    # Caso de múltiplas avaliações
                    st.markdown('<p class="subtitle">Resultados:</p>', unsafe_allow_html=True)
                    
                    # Criar DataFrame para exibir resultados
                    df_results = pd.DataFrame(results, columns=['Avaliação', 'Sentimento'])
                    
                    # Contar ocorrências de cada sentimento
                    sentiment_counts = df_results['Sentimento'].value_counts()
                    
                    # Garantir que todos os sentimentos estejam representados
                    for sentiment in ['Positivo', 'Neutro', 'Negativo']:
                        if sentiment not in sentiment_counts:
                            sentiment_counts[sentiment] = 0
                    
                    # Calcular percentuais
                    total = len(results)
                    sentiment_percentages = (sentiment_counts / total * 100).reindex(['Positivo', 'Neutro', 'Negativo'])
                    
                    # Exibir tabela de resultados
                    st.dataframe(df_results, use_container_width=True)
                    
                    # Plotar gráfico de barras com percentuais
                    fig, ax = plt.subplots(figsize=(10, 5))
                    fig.patch.set_facecolor('#1E1E1E')
                    ax.set_facecolor('#1E1E1E')
                    
                    bars = ax.bar(['Positivo', 'Neutro', 'Negativo'], 
                                sentiment_percentages, 
                                color=['#4CAF50', '#9E9E9E', '#F44336'])
                    
                    # Adicionar rótulos e título
                    ax.set_ylabel('Percentual (%)', color='white')
                    ax.set_title('Distribuição de Sentimentos', color='white')
                    
                    # Personalizar eixos
                    ax.spines['bottom'].set_color('white')
                    ax.spines['top'].set_color('white')
                    ax.spines['right'].set_color('white')
                    ax.spines['left'].set_color('white')
                    ax.tick_params(axis='x', colors='white')
                    ax.tick_params(axis='y', colors='white')
                    
                    # Adicionar valores nas barras
                    for bar in bars:
                        height = bar.get_height()
                        ax.annotate(f'{height:.1f}%',
                                    xy=(bar.get_x() + bar.get_width() / 2, height),
                                    xytext=(0, 3),
                                    textcoords="offset points",
                                    ha='center', va='bottom',
                                    color='white')
                    
                    st.pyplot(fig)
            else:
                st.warning("Por favor, digite uma avaliação de vinho para classificar.")
    
    with tab2:
        st.markdown('<p class="subtitle">Sobre o WineMood</p>', unsafe_allow_html=True)
        
        st.markdown("""
        ## O que é o WineMood?
        
        WineMood é um sistema inteligente que analisa avaliações textuais de vinhos e classifica automaticamente o sentimento expresso como positivo, neutro ou negativo. Utilizando técnicas de Processamento de Linguagem Natural (NLP) combinadas com Deep Learning, o projeto transforma a linguagem humana em insights úteis para consumidores e empresas.
        
        ## Como funciona?
        
        O sistema utiliza um modelo de Deep Learning baseado em redes neurais recorrentes (LSTM) para entender o contexto e o sentimento das avaliações de vinhos. O processo inclui:
        
        1. **Pré-processamento de texto**: limpeza, remoção de stopwords e tokenização
        2. **Vetorização**: conversão de palavras em representações numéricas
        3. **Análise de sequência**: compreensão do contexto através de LSTM
        4. **Classificação**: determinação do sentimento predominante
        
        ## Aplicações práticas
        
        - Recomendação automática de vinhos com base no sentimento do cliente
        - Auxílio a vendedores e produtores de vinho para entender a percepção do público
        - Destaque de vinhos bem avaliados em plataformas de e-commerce
        
        ## Tecnologias utilizadas
        
        - **Python**: linguagem de programação principal
        - **TensorFlow + Keras**: framework de Deep Learning
        - **NLTK**: biblioteca para processamento de linguagem natural
        - **Streamlit**: interface web interativa
        - **Pandas & NumPy**: manipulação de dados
        - **Matplotlib**: visualização de dados
        
        ## Dataset
        
        O modelo foi treinado com aproximadamente 130.000 avaliações de vinhos do Kaggle, com pontuações convertidas em categorias de sentimento:
        
        - **Positivo**: pontuação ≥ 90
        - **Neutro**: pontuação entre 80-89
        - **Negativo**: pontuação < 80
        """)

if __name__ == "__main__":
    main()

## 12. Instruções para Deploy

### Deploy Local

Para executar a aplicação localmente:

1. Instale as dependências: `pip install -r requirements.txt`
2. Execute o Streamlit: `streamlit run app.py`

### Deploy no Streamlit Cloud

Para fazer o deploy no Streamlit Cloud:

1. Crie uma conta no [Streamlit Cloud](https://streamlit.io/cloud)
2. Crie um repositório no GitHub com os seguintes arquivos:
   - app.py
   - requirements.txt
   - model.h5
   - tokenizer.pickle
3. Conecte o repositório ao Streamlit Cloud
4. Configure o deploy com o arquivo principal como app.py

## 13. Conclusão

Neste notebook, desenvolvemos um classificador de sentimentos para avaliações de vinhos usando Deep Learning. O modelo alcançou uma boa acurácia e foi implementado em uma interface web interativa usando Streamlit.

O projeto WineMood demonstra como técnicas de NLP e Deep Learning podem ser aplicadas a dados do mundo real para extrair insights valiosos e criar aplicações práticas para consumidores e empresas.